# Sea Ice Drift

In [1]:
#directory paths
_work_dir='/g/data/jk72/as2285/miz/'
_data_dir='/g/data/jk72/MIZ/'

In [2]:
#some constants
CLIMAT_DATES=[1981,2010]
EAST_ANT_LONS=[71,160] #longitudes for east Antartica (easterly)
YEAR=2021
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
#TO-DO - move file paths here?

In [3]:
#useful py libraries
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.ticker as ticker

In [4]:
# import local functions
import sys
sys.path.append(_work_dir)
from utils.climat import climatology

%run {_work_dir}utils/plot_tools.ipynb

In [5]:
%matplotlib inline

In [6]:
# The Dataset

In [7]:
files = !ls -d {_data_dir}/OSI-SAF/drift_lr/*.nc

In [ ]:
osi_motion_ds=xr.open_mfdataset(files)

In [ ]:
osi_motion_ds.xc=osi_motion_ds.xc*1000
osi_motion_ds.yc=osi_motion_ds.yc*1000

In [ ]:
motion_mean_dx_ds=climatology(osi_motion_ds.dX, climat_dates=None).rename({'ave':'dX_ave','st_dev':'dX_st_dev'})
motion_mean_dy_ds=climatology(osi_motion_ds.dY, climat_dates=None).rename({'ave':'dY_ave','st_dev':'dY_st_dev'})

motion_mean_ds=xr.merge([motion_mean_dx_ds,motion_mean_dy_ds])

In [ ]:
motion_anoms_dx_da=osi_motion_ds.dX.groupby('time.month')-motion_mean_dx_ds.ave
motion_anoms_dy_da=osi_motion_ds.dY.groupby('time.month')-motion_mean_dy_ds.ave


motion_anoms_ds=xr.merge([motion_anoms_dx_da,motion_anoms_dy_da])

In [ ]:
motion_mean_ds

In [ ]:
motion_mean_ds.load()

In [ ]:
for MONTH in np.arange(1,13,1):

    to_plot=motion_mean_ds.sel(month=MONTH)
    
    plt.figure(figsize=(12,12))
    ax=plt.subplot(projection=ccrs.SouthPolarStereo(true_scale_latitude=-70))

    plt.quiver(
        (to_plot.xc*1000), 
        (to_plot.yc*1000), 
        to_plot.dX_ave,
        to_plot.dY_ave,
        transform=ccrs.SouthPolarStereo(true_scale_latitude=-70)
    )

    ax.coastlines()

    #ax.set_extent([EAST_ANT_LONS[0], EAST_ANT_LONS[1], -75, -60], crs=ccrs.PlateCarree())

    add_south_pole_gridlines(ax)
    
    plt.title(f'Motion {months[MONTH-1]} average')
    
    plt.show()
    
    